In [3]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import joblib
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#----------------------------------------------------------------------------------
file = os.path.join('C:/Users/eamo8/Desktop/Notebook para repositorio/Reboiler Raw Data', 'Reboiler_Depurado.csv')
df12 = pd.read_csv(file, header=0)
attrs = {
	'var': 'df12',
	'type': type(df12).__name__,
}
[attrs]
df12
#----------------------------------------------------------------------------------
df12 = df12
s = df12['Fecha']
df12['Fecha'] = pd.to_datetime(s, errors='coerce')

s = df12['Conductividad']
df12['Conductividad'] = pd.to_numeric(s, errors='coerce')

s = df12['Carga Motor (M014)']
df12['Carga Motor (M014)'] = pd.to_numeric(s, errors='coerce')

s = df12['Carga Motor (M015)']
df12['Carga Motor (M015)'] = pd.to_numeric(s, errors='coerce')

s = df12['Ind.Presión (PI8026)']
df12['Ind.Presión (PI8026)'] = pd.to_numeric(s, errors='coerce')

s = df12['Ind.Presión (PI8025)']
df12['Ind.Presión (PI8025)'] = pd.to_numeric(s, errors='coerce')

s = df12['Ind.Temperatura (TI8015)']
df12['Ind.Temperatura (TI8015)'] = pd.to_numeric(s, errors='coerce')

s = df12['Ind.Temperatura (TI8014)']
df12['Ind.Temperatura (TI8014)'] = pd.to_numeric(s, errors='coerce')

s = df12['Estado']
df12['Estado'] = pd.to_numeric(s, errors='coerce')

attrs = {
	'var': 'df12',
	'type': type(df12).__name__,
}
[attrs]
df12
#----------------------------------------------------------------------------------
df12 = df12
df12.set_index([
	'Fecha',
], inplace=True)
attrs = {
	'var': 'df12',
	'type': type(df12).__name__,
}
[attrs]
df12
#----------------------------------------------------------------------------------
df24 = df12.drop([
	'Estado',
], axis='columns')
attrs = {
	'var': 'df24',
	'type': type(df24).__name__,
}
[attrs]
df24
#----------------------------------------------------------------------------------
df16 = df12[[
	'Estado',
]]
attrs = {
	'var': 'df16',
	'type': type(df16).__name__,
}
[attrs]
df16
#----------------------------------------------------------------------------------
def datetime_window(dfx, dfy, dfy_total, scaler, size, periods, y_choose='last', ahead=''):
    catcolsx = dfx.select_dtypes(['category']).columns
    dfx[catcolsx] = dfx[catcolsx].apply(lambda x: x.cat.codes)

    catcolsy = dfy.select_dtypes(['category']).columns
    dfy[catcolsy] = dfy[catcolsy].apply(lambda x: x.cat.codes)

    catcolsy_total = dfy_total.select_dtypes(['category']).columns
    dfy_total[catcolsy_total] = dfy_total[catcolsy_total].apply(lambda x: x.cat.codes)

    ind = []
    x = []
    y = []

    def get_windows(win):
        if (win.shape[0] != periods):
            return np.nan
        
        if (y_choose == 'ahead'):
            wy = dfy.loc[win.index, :]
            try:
                wy = dfy_total.loc[wy.index[-1] + pd.Timedelta(ahead)]
                y.append(wy)

                wx = dfx.loc[win.index, :]
                x.append(wx)

                ind.append(win.index[0])
            except:
                pass
        else:
            wx = dfx.loc[win.index, :]
            x.append(wx)

            wy = dfy.loc[win.index, :]
            if (y_choose == 'last'):
                wy = wy.iloc[-1]
            elif (y_choose == 'first'):
                wy = wy.iloc[0]
            y.append(wy)

            ind.append(win.index[0])
        
        return np.nan

    dfx.iloc[:, 0].rolling(size).apply(get_windows)
    
    return x, y, ind

# --------------------------------------------------------------------------------------
def print_list_shape(lst):
    s = ''
    if (len(lst) > 0):
        s += str(len(lst))
        s += ' x ' + str(np.array(lst[0]).shape)
    else:
        s += 'empty'
        
    return s

# --------------------------------------------------------------------------------------
def normal_anomal_split(dfx, dfy, size, periods, y_choose='last', ahead='', custom_normal_split=False, normal_test_ratio=0.1, scaler='MinMaxScaler'):
    
    if (scaler == 'MinMaxScaler'):
        myscaler = MinMaxScaler(feature_range=(0,1))
    elif (scaler == 'StandardScaler'):
        myscaler = StandardScaler()
    elif(scaler == 'RobustScaler'):
        myscaler = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(25, 75))
    
    x_windowed, y_windowed, windows_ind = datetime_window(dfx=dfx, dfy=dfy, dfy_total=dfy, scaler=myscaler, size=size, periods=periods, y_choose=y_choose, ahead=ahead)
    
    
    x_normal = []
    y_normal = []
    x_anomal = []
    y_anomal = []
    for (i, wy) in enumerate(y_windowed):
        if wy.iloc[0] == 0:
            x_normal.append(x_windowed[i])
            y_normal.append(y_windowed[i])
        else:
            x_anomal.append(x_windowed[i])
            y_anomal.append(y_windowed[i])
    
    
    normal_indices = [i for i in range(len(x_normal))]
    normal_indices_permuted = np.random.permutation(normal_indices)
    
    length = len(x_normal)
    length_test = round(length * normal_test_ratio)
    
    if custom_normal_split:
        normal_test_indices = normal_indices_permuted[:length_test]
        normal_train_indices = normal_indices_permuted[length_test:]
    else:
        normal_test_indices = normal_indices_permuted[:len(y_anomal)]
        normal_train_indices = normal_indices_permuted[len(y_anomal):]
    
    
    x_normal_train = [x_normal[i] for i in normal_train_indices]
    x_normal_test = [x_normal[i] for i in normal_test_indices]
    
    
    x_normal_train_total = pd.concat(x_normal_train)
    x_normal_train_total = x_normal_train_total[~x_normal_train_total.index.duplicated(keep='first')]
    x_normal_train_total.sort_index(inplace=True)
    
    myscaler.fit(x_normal_train_total.to_numpy())
    
    x_train = []
    for wx in x_normal_train:
        wx = wx.to_numpy()
        wx = np.squeeze(myscaler.transform(wx))
        x_train.append(wx)
    
    
    x_test = []
    y_test = []
    for wx in x_normal_test:
        wx = wx.to_numpy()
        wx = np.squeeze(myscaler.transform(wx))
        x_test.append(wx)
        y_test.append(0)
        
    for wx in x_anomal:
        wx = wx.to_numpy()
        wx = np.squeeze(myscaler.transform(wx))
        x_test.append(wx)
        y_test.append(1)
    
    x_mix = []
    for wx in x_windowed:
        wx = wx.to_numpy()
        wx = np.squeeze(myscaler.transform(wx))
        x_mix.append(wx)
    
    y_mix = []
    for wy in y_windowed:
        wy = wy.to_numpy()
        wy = wy[0]
        y_mix.append(wy)
        
    i_mix = windows_ind

    
    return x_train, x_test, y_test, x_mix, y_mix, i_mix, myscaler
x_train, x_test, y_test, x_mix, y_mix, i_mix, myscaler = normal_anomal_split(
	dfx=df24, dfy=df16
	, size='48h', periods=24, y_choose='last', ahead='60T'
	, custom_normal_split=False, normal_test_ratio=0.2, scaler='MinMaxScaler')

df24 = {
	'x_train': np.array(x_train),
	'x_test': np.array(x_test),
	'y_test': np.array(y_test),
	'x_mix': np.array(x_mix),
	'y_mix': np.array(y_mix),
	'i_mix': i_mix,
	'myscaler': myscaler,
}

attrs = {
	'var': 'df24',
	'type': type(df24).__name__,
}
[attrs]

{
	'x_train': print_list_shape(x_train),
	'x_test': print_list_shape(x_test),
	'y_test': print_list_shape(y_test),
}

np.squeeze(df24['x_train'])
np.squeeze(df24['x_test'])
np.squeeze(df24['y_test'])
# --------------------------------------------------------------------------------------
file = os.path.join('C:/Users/eamo8/Desktop/Notebook para repositorio/Reboiler Raw Data', 'Ventanas de 48 horas.npz')
np.savez(file, x_train=df24['x_train'], x_test=df24['x_test'], y_test=df24['y_test'], x_mix=df24['x_mix'], y_mix=df24['y_mix'], i_mix=df24['i_mix'])
scaler_file = os.path.join('C:/Users/eamo8/Desktop/Notebook para repositorio/Reboiler Raw Data', 'scaler.joblib')
joblib.dump(df24['myscaler'], scaler_file)
attrs = {
	'var': 'empty',
	'type': 'empty',
}
[attrs]
'npz file saved'


[{'var': 'df12', 'type': 'DataFrame'}]

,Fecha,Conductividad,Carga Motor (M014),Carga Motor (M015),Ind.Presión (PI8026),Ind.Presión (PI8025),Ind.Temperatura (TI8015),Ind.Temperatura (TI8014),Estado
0,2016-01-08 10:00:00,4.5,69.658713,42.280192,18.769251,19.0,79.399877,99.160674,0.0
1,2016-01-08 12:00:00,4.4,67.954487,42.381221,18.411020,19.0,78.184952,98.775669,0.0
2,2016-01-08 14:00:00,4.2,69.688606,41.967134,18.601041,19.0,80.599028,101.165308,0.0
3,2016-01-08 16:00:00,4.2,69.274182,42.124866,18.582166,19.0,79.301819,99.934365,0.0
4,2016-01-08 18:00:00,4.2,69.095421,41.588124,18.593953,19.0,80.170472,100.804536,0.0
...,...,...,...,...,...,...,...,...,...
11161,2018-09-06 00:00:00,7.4,72.696510,43.464317,19.695450,19.5,81.372489,99.598170,0.0
11162,2018-09-06 02:00:00,7.2,72.905098,44.058929,19.826046,19.5,81.911505,99.633708,0.0
11163,2018-09-06 04:00:00,6.9,72.968914,44.584087,19.523726,19.5,80.810454,99.775918,0.0
11164,2018-09-06 06:00:00,6.9,72.905579,44.417335,19.308117,19.5,81.587220,100.136055,0.0


[{'var': 'df12', 'type': 'DataFrame'}]

,Fecha,Conductividad,Carga Motor (M014),Carga Motor (M015),Ind.Presión (PI8026),Ind.Presión (PI8025),Ind.Temperatura (TI8015),Ind.Temperatura (TI8014),Estado
0,2016-01-08 10:00:00,4.5,69.658713,42.280192,18.769251,19.0,79.399877,99.160674,0.0
1,2016-01-08 12:00:00,4.4,67.954487,42.381221,18.411020,19.0,78.184952,98.775669,0.0
2,2016-01-08 14:00:00,4.2,69.688606,41.967134,18.601041,19.0,80.599028,101.165308,0.0
3,2016-01-08 16:00:00,4.2,69.274182,42.124866,18.582166,19.0,79.301819,99.934365,0.0
4,2016-01-08 18:00:00,4.2,69.095421,41.588124,18.593953,19.0,80.170472,100.804536,0.0
...,...,...,...,...,...,...,...,...,...
11161,2018-09-06 00:00:00,7.4,72.696510,43.464317,19.695450,19.5,81.372489,99.598170,0.0
11162,2018-09-06 02:00:00,7.2,72.905098,44.058929,19.826046,19.5,81.911505,99.633708,0.0
11163,2018-09-06 04:00:00,6.9,72.968914,44.584087,19.523726,19.5,80.810454,99.775918,0.0
11164,2018-09-06 06:00:00,6.9,72.905579,44.417335,19.308117,19.5,81.587220,100.136055,0.0


[{'var': 'df12', 'type': 'DataFrame'}]

...

array([[[0.17647059, 0.94107379, 0.7864206 , ..., 0.06429392,
         0.66567049, 0.60757351],
        [0.17647059, 0.96458606, 0.78578977, ..., 0.06429392,
         0.68997309, 0.60606309],
        [0.17647059, 0.95903065, 0.81057494, ..., 0.06429392,
         0.67870533, 0.60569128],
        ...,
        [0.20168067, 0.95511918, 0.7906398 , ..., 0.05855339,
         0.6908691 , 0.60336497],
        [0.19327731, 0.95630569, 0.79941614, ..., 0.05855339,
         0.69326768, 0.60558348],
        [0.19327731, 0.95405435, 0.79877762, ..., 0.05855339,
         0.69073483, 0.61243352]],

       [[0.33613445, 0.93700334, 0.83521056, ..., 0.0413318 ,
         0.65425661, 0.60585781],
        [0.33613445, 0.94763286, 0.83082458, ..., 0.0413318 ,
         0.67496607, 0.60267337],
        [0.34453782, 0.95575499, 0.80971338, ..., 0.0413318 ,
         0.6601212 , 0.59905688],
        ...,
        [0.33613445, 0.95608718, 0.77529026, ..., 0.05051665,
         0.68467642, 0.60169143],
        [0.3

array([0, 0, 0, ..., 1, 1, 1], shape=(4612,))

['C:/Users/eamo8/Desktop/Notebook para repositorio/Reboiler Raw Data\\scaler.joblib']

[{'var': 'empty', 'type': 'empty'}]

'npz file saved'